In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import copy
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

import torchvision.models as models

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()


In [32]:
LEARNING_RATE = 0.0001
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)

#model = models.resnet18(pretrained=True)
print(model)

# Number of classes in the dataset
num_classes = 10
#model.classifier[6] = nn.Linear(4096,num_classes)
model.fc = nn.Linear(1000,num_classes)
print(model)


Using cache found in C:\Users\Arvid/.cache\torch\hub\pytorch_vision_v0.10.0


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [25]:
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs):

    #print('hit kom jag 2.')
    train_losses = []
    valid_losses = []
    best_model_loss = 10
    
    for epoch in range(num_epochs):

        # Training
        for i, (data, labels) in enumerate(train_loader):
            
            prediction = model.forward(data)

            train_loss = criterion(prediction, labels)

            train_loss.backward()

            optimizer.step()

            optimizer.zero_grad()
        print(
            f'\rEpoch {epoch+1}, batch {i+1}/{len(train_loader)} - Loss: {train_loss}',"\n"
        )

        train_losses.append(train_loss)
        writer.add_scalar("Loss/train ADAM", train_loss, epoch)

        # Validation
        print('hit kom jag 5.')
        for batch_nr, (data, labels) in enumerate(val_loader):
            prediction = model.forward(data)
            loss_val = criterion(prediction, labels)
            valid_losses.append(loss_val)
        print(f"loss validation: {loss_val}","\n")

        if valid_losses[-1] < best_model_loss:
            print(f"\t > Found a better model, {best_model_loss} -> {valid_losses[-1]}")
            best_model = copy.deepcopy(model)
            best_model_loss = valid_losses[-1]

        writer.add_scalar("Loss/validation ADAM", loss_val, epoch)

    print(f"\nBest model loss: {best_model_loss}")
    return best_model, train_losses, valid_losses

def get_accuracy(network, loader):
    
    with torch.no_grad():
        correct = 0
        total = 0
        y_pred = []
        y_true = []

        for x, (data, labels) in enumerate(loader):

            prediction = network.forward(data)

            for i in range(len(data)):

                y_true.append(labels[i].item())
                y_pred.append(torch.argmax(prediction[i]).item())
                if y_true[i] == y_pred[i]:
                    correct += 1        
    
            total += len(data)
    
        score = correct/total

        accuracy = score

        return accuracy
    

In [33]:
LEARNING_RATE = 0.0001
EPOCHS = 3
BATCH_SIZE = 10

#transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform)

validset, trainset = torch.utils.data.random_split(trainset, [49900, 100])

trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,shuffle=True)
validloader = torch.utils.data.DataLoader(validset, batch_size=BATCH_SIZE,shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,shuffle=False)

# Define our loss function
criterion = nn.CrossEntropyLoss()

# Define our optimizer
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

print('hit kom jag 1.')
# Train the model
trained_model, train_loss, valid_loss = train_model(model, criterion, optimizer, trainloader, validloader, EPOCHS)

# Test the model
test_acc = get_accuracy(trained_model, testloader)

writer.flush()

Files already downloaded and verified
Files already downloaded and verified
hit kom jag 1.
Epoch 1, batch 10/10 - Loss: 5.25208854675293 

hit kom jag 5.


KeyboardInterrupt: 

In [49]:
model_fe = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
model_fe.fc = nn.Linear(1000,num_classes)
print(model_fe.named_parameters())
print(model_fe)
# Freeze all layers except the last few layers
for name, param in model_fe.named_parameters():
    if "classifier.6" in name or "classifier.4" in name or "fc" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

# Print which layers will be updated during fine-tuning
params_to_update = []
for name, param in model_fe.named_parameters():
    if param.requires_grad:
        params_to_update.append(param)
        print("\t", name)


LEARNING_RATE = 0.0001
EPOCHS = 3
BATCH_SIZE = 10

#transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform)

validset, trainset = torch.utils.data.random_split(trainset, [49900, 100])

trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,shuffle=True)
validloader = torch.utils.data.DataLoader(validset, batch_size=BATCH_SIZE,shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,shuffle=False)

# Define our loss function
criterion = nn.CrossEntropyLoss()

# Define our optimizer
optimizer = torch.optim.Adam(model_fe.parameters(), lr = LEARNING_RATE)

# Train the model
trained_model, train_loss, valid_loss = train_model(model_fe, criterion, optimizer, trainloader, validloader, EPOCHS)

# Test the model
test_acc = get_accuracy(trained_model, testloader)

writer.flush()

Using cache found in C:\Users\Arvid/.cache\torch\hub\pytorch_vision_v0.10.0
c:\Users\Arvid\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Arvid\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<generator object Module.named_parameters at 0x000001F7CD091CB0>
AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)


KeyboardInterrupt: 